<a href="https://colab.research.google.com/github/a-winders/MLA_ATW/blob/main/MLA2_ATW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Assignment 2
# Adam Winders 
# CS430-ON 
# Machine Learning in the Cloud

<br>
<br>
<br>

## Standard Imports

In [131]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [132]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Imports for specific models
In this section, you will import the models from SKLearn that you will use this in this notebook. <br> <br>
**Include imports for both Linear Regression and Logistic Regression**

In [189]:
# Imports LinearRegression & LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

Linear Regression - Acquire data

In [213]:
df = pd.read_csv('/content/gdrive/MyDrive/Bellarmine/CS430_Data_Files/Steel_industry_data.csv')

## Imports for Splitting Data and Building Pipeline


In [194]:
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn import set_config
set_config(display='diagram')

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

## Imports for Performance Measures

In [195]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc

## Confusion Matrix Function
This function is for creating a graphical version of the confusion matrix.



In [196]:
def plot_cm(y_test, y_pred):
    cm = confusion_matrix(y_test,y_pred)
    fig = plt.figure(figsize=(10,10))
    heatmap = sns.heatmap(cm, annot=True, fmt='.2f', cmap='RdYlGn')
    plt.ylabel('True label')
    plt.xlabel('Predicted Label')

## ROC_Curve Function
This function is for plotting the roc_curve

In [197]:
def plot_roc_curve(fpr,tpr):
  plt.plot(fpr, tpr, linewidth=2)
  plt.plot([0,1],[0,1], 'k--')
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.0])

<br>
<br>

# **Problem 1**

Renames the columns according to **Table 1: Columns Names**

In [212]:
# displays old column names with first 5 rows
df.head()

,date,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,01/01/2018 00:45,3.24,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,01/01/2018 01:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,01/01/2018 01:15,3.82,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load


In [214]:
# renames columns
df.rename(columns=
          {'Usage_kWh':'usage_kwh'
          ,'Lagging_Current_Reactive.Power_kVarh':'lag_react_pwr_kvarh'
          ,'Leading_Current_Reactive_Power_kVarh':'lead_react_pwr_kvarh'
          ,'Lagging_Current_Power_Factor':'lag_current_pwr'
          ,'Leading_Current_Power_Factor':'lead_current_pwr'
          ,'NSM':'nsm'
          ,'WeekStatus':'week_status'
          ,'Day_of_week':'day_of_week'
          ,'Load_Type':'load_type'
           }, inplace = True)


In [207]:
# displays new column names
df.head()

,date,usage_kwh,lag_react_pwr_kvarh,lead_react_pwr_kvarh,CO2(tCO2),lag_current_pwr,lead_current_pwr,nsm,week_status,day_of_week,load_type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,01/01/2018 00:45,3.24,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,01/01/2018 01:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,01/01/2018 01:15,3.82,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load


##Statistics and Visual Exploration

Summarize the numerical columns of data

In [188]:
df.describe()
#df.select_dtypes(include=np.number)

,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM
count,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000
mean,27.386892,13.035384,3.870949,0.011524,80.578056,84.367870,42750.000000
std,33.444380,16.306000,7.424463,0.016151,18.921322,30.456535,24940.534317
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.200000,2.300000,0.000000,0.000000,63.320000,99.700000,21375.000000
50%,4.570000,5.000000,0.000000,0.000000,87.960000,100.000000,42750.000000
75%,51.237500,22.640000,2.090000,0.020000,99.022500,100.000000,64125.000000
max,157.180000,96.910000,27.760000,0.070000,100.000000,100.000000,85500.000000


Get the counts of the different load types. Save this into a data frame so you can plot it.

In [143]:
# displays data types of each column
df.dtypes

date                     object
usage_kwh               float64
lag_react_pwr_kvarh     float64
lead_react_pwr_kvarh    float64
CO2(tCO2)               float64
lag_current_pwr         float64
lead_current_pwr        float64
nsm                       int64
week_status              object
day_of_week              object
load_type                object
dtype: object

In [144]:
load_type= df['load_type'].groupby(df['load_type']).count()

In [145]:
load_type

load_type
Light_Load      18072
Maximum_Load     7272
Medium_Load      9696
Name: load_type, dtype: int64

Create a barplot to view how many days fall under each load type. The size of the plot should be 10 x 5.

In [146]:
"""
df.plot.bar(x='load_type',
            rot=0,
            figsize=(10,5));
"""

"\ndf.plot.bar(x='load_type',\n            rot=0,\n            figsize=(10,5));\n"

Create a pairplot of all of the numerical variables using Seaborn

In a markdown comment - write up any interesting patterns you see from the pairplot

**TODO**

## Splitting Data into Train/Test

Create a data frame called X of your inputs that includes all columns except data and usage_kwh
<br>
<br>
Create a data frame called y of your output that includes only usage_kwh. This will be your target variable that you are trying to predict.

In [35]:
X = df[['lag_react_pwr_kvarh','lead_react_pwr_kvarh','lag_current_pwr','lead_current_pwr','nsm','week_status','day_of_week','load_type']]
y = df['usage_kwh']

Split the data into Train/Test sets. Use 20% of the data for testing. You should have 4 data frames: **X_train, X_test, y_train, y_test**

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)

In [54]:
type(X_train)

pandas.core.frame.DataFrame

In [55]:
X_train = np.array(X_train).reshape(-1,1)
X_test = np.array(X_test).reshape(-1,1)

In [56]:
lr_model = LinearRegression()
lr_model.fit(X_train,y_train)

ValueError: ignored

## Building the Pipeline

Create a column transformer that will OneHotEncode the categorical variables and MinMax scale the numerical variables (not the output)

In [57]:
df.head()

,date,usage_kwh,lag_react_pwr_kvarh,lead_react_pwr_kvarh,CO2(tCO2),lag_current_pwr,lead_current_pwr,nsm,week_status,day_of_week,load_type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,01/01/2018 00:45,3.24,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,01/01/2018 01:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,01/01/2018 01:15,3.82,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load


In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)

In [117]:
num_attribs = ['lag_react_pwr_kvarh','lead_react_pwr_kvarh','lag_current_pwr','lead_current_pwr','nsm','week_status','day_of_week','load_type']
cat_attribs = ['usage_kwh']

Create a pipeline that will transform the data and then run a Linear Regression

In [147]:
col_transform = ColumnTransformer(transformers=[
                               ('cat',OneHotEncoder(), cat_attribs),
                               ('num',MinMaxScaler(), num_attribs),]
                  )

In [148]:
pipeline = Pipeline([
                      ('transform',col_transform),
                      ('mlr_model',LinearRegression())
])

In [149]:
pipeline

Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  ['day_of_week', 'load_type']),
                                                 ('num', MinMaxScaler(),
                                                  ['date', 'usage_kwh',
                                                   'lag_react_pwr_kvarh',
                                                   'lead_react_pwr_kvarh',
                                                   'CO2(tCO2)',
                                                   'lag_current_pwr',
                                                   'lead_current_pwr',
                                                   'nsm'])])),
                ('mlr_model', LinearRegression())])

## Executing the Model

Run the pipeline to fit the Linear Regression Model

In [150]:
pipeline.fit(X_train,y_train)

ValueError: ignored

## Evaluating the Model

Run the pipeline to predict y values from the test data set and save them in a variable called **y_pred**

In [64]:
y_pred = pipeline.predict(X_test)

AttributeError: ignored

Calcuate the root mean squared error for both train and test. In a markdown comment below the result, interpret the output you receive.

Calculate the R-squared value of the model (test only). In a markdown comment below the result, interpret the output you receive.

In [65]:
r2_score(y_pred, y_test)

NameError: ignored

Write the code to display the intercept value (bias) and coefficients of the model. **(These will be two different lines of code).**

In [66]:
pipeline.named_steps['mlr_model'].intercept_

AttributeError: ignored

In [ ]:
pipeline.named_steps['mlr_model'].coef_

## Conclusion

In a markdown comment, write a conclusion on how suitable you think this model is for predicting kwh usage.

<br>
<br>

# **Problem 2**

## Reading the data

 Recodes the 'week_status' column so a Weekday is coded as 1 and a Weekend is coded as a 0.

In [250]:
# displays week_status column before conversion
weekday_weekend = df[["week_status","day_of_week"]]
weekday_weekend

,week_status,day_of_week
0,0,Monday
1,0,Monday
2,0,Monday
3,0,Monday
4,0,Monday
...,...,...
35035,0,Monday
35036,0,Monday
35037,0,Monday
35038,0,Monday


In [318]:
df['week_status'] = df['week_status'].apply(lambda x: 1 if x == "Weekday" else 0)

In [319]:
df.day_of_week.groupby(df.day_of_week).count()

day_of_week
Friday       4992
Monday       5088
Saturday     4992
Sunday       4992
Thursday     4992
Tuesday      4992
Wednesday    4992
Name: day_of_week, dtype: int64

## Splitting Data into Train/Test

In [320]:
df.head()

,date,usage_kwh,lag_react_pwr_kvarh,lead_react_pwr_kvarh,CO2(tCO2),lag_current_pwr,lead_current_pwr,nsm,week_status,day_of_week,load_type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,0,Monday,Light_Load
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,0,Monday,Light_Load
2,01/01/2018 00:45,3.24,3.28,0.0,0.0,70.28,100.0,2700,0,Monday,Light_Load
3,01/01/2018 01:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,0,Monday,Light_Load
4,01/01/2018 01:15,3.82,4.50,0.0,0.0,64.72,100.0,4500,0,Monday,Light_Load


In [331]:
X = df[['usage_kwh','lag_react_pwr_kvarh', 'lag_react_pwr_kvarh','lead_react_pwr_kvarh','CO2(tCO2)','lag_current_pwr','lead_current_pwr','nsm','load_type']]
y = df['week_status']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.20, random_state=32)

## Building the Pipeline

Creates a column transformer that will OneHotEncode the categorical variables and MinMax scale the numerical variables

In [338]:
df.head()

,date,usage_kwh,lag_react_pwr_kvarh,lead_react_pwr_kvarh,CO2(tCO2),lag_current_pwr,lead_current_pwr,nsm,week_status,day_of_week,load_type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,0,Monday,Light_Load
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,0,Monday,Light_Load
2,01/01/2018 00:45,3.24,3.28,0.0,0.0,70.28,100.0,2700,0,Monday,Light_Load
3,01/01/2018 01:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,0,Monday,Light_Load
4,01/01/2018 01:15,3.82,4.50,0.0,0.0,64.72,100.0,4500,0,Monday,Light_Load


In [339]:
num_attribs = ['usage_kwh','lag_react_pwr_kvarh','lead_react_pwr_kvarh','CO2(tCO2)','lag_current_pwr','lead_current_pwr','nsm']
cat_attribs = ['day_of_week','load_type']

In [340]:
col_transform = ColumnTransformer(transformers=[
                               ('cat',OneHotEncoder(), cat_attribs),
                               ('num',MinMaxScaler(), num_attribs),]
                  )

Creates a pipeline that will transform the data and then run a Logistic Regression

In [347]:
pipeline = Pipeline([
                     ('trans', col_transform),
                     ('mlr', LogisticRegression(max_iter=1000)),
                     ])

In [348]:
from sklearn import set_config
set_config(display='diagram')

In [349]:
pipeline

Pipeline(steps=[('trans',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  ['day_of_week', 'load_type']),
                                                 ('num', MinMaxScaler(),
                                                  ['usage_kwh',
                                                   'lag_react_pwr_kvarh',
                                                   'lead_react_pwr_kvarh',
                                                   'CO2(tCO2)',
                                                   'lag_current_pwr',
                                                   'lead_current_pwr',
                                                   'nsm'])])),
                ('mlr', LogisticRegression(max_iter=1000))])

## Executing the Model

In [354]:
pipeline.fit(X_train, np.ravel(y_train))

ValueError: ignored

## Evaluating the Model

Predicts y values from the test data set and saves them in a variable called y_pred

In [ ]:
y_pred = pipeline.predict(X_test)

Calls the plot_cm function to generate the confusion matrix

In [ ]:
plot_cm(y_test,y_pred)

Calculates the precision, recall, and F1 score

In [ ]:
print("Precision: ", precision_score(y_test,y_pred)) 
print("Recall: ", recall_score(y_test,y_pred))
print("F1 Score: ", f1_score(y_test,y_pred))

**Results**
<br>
As of now, the algorithm will not run at **pipeline.fit(X_train, np.ravel(y_train))** because:
<br>
"A given column is not a column of the dataframe"

Generates an ROC Curve, or a Precision-Recall Curve, as appropriate with (AUC)

In [ ]:
y_pred_proba = pipeline.predict_proba(X_test)[::,1]
fpr, tpr, _ = roc_curve(y_test,y_pred_proba)
plot_roc_curve(fpr, tpr)
print(roc_auc_score(y_test, y_pred_proba))

## Conclusion
<br>
This model is **BLANK** for predicting week_status. According to the array produced by y_pred_prob, the column representing the probability of the positive class shows...

Write a general conclusion about the results of your model